<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/SS_0321_Ensemble_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___


직접투표 : 다수결

간접 투표 : 개별 분류기가 예측하는 확률을 평균내어 그 값이 가장 높은 높은 클래스로 예측

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

낮추어야함



### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [3]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [5]:
# model fitting (25분 소요)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
ex = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm = LinearSVC()
mlp = MLPClassifier()

rf.fit(X_train, y_train)
ex.fit(X_train, y_train)
svm.fit(X_train, y_train)
mlp.fit(X_train, y_train)

MLPClassifier()

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [6]:
from sklearn.ensemble import VotingClassifier

In [7]:
# model fitting (28분 소요)

voting = VotingClassifier(estimators=[('et', ex), ('rf', rf), ('svc', svm), ('mlp',mlp)],voting = 'hard')
voting.fit(X_train, y_train)

VotingClassifier(estimators=[('et', ExtraTreesClassifier(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', LinearSVC()), ('mlp', MLPClassifier())])

In [8]:
# model test
voting.score(X_val, y_val)

0.9684821428571428

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [9]:
# 각 분류기의 검증 데이터셋 정확도 확인
for clf in (ex, rf, svm, mlp):
    print(clf.__class__.__name__, clf.score(X_val, y_val))

ExtraTreesClassifier 0.9707142857142858
RandomForestClassifier 0.9658928571428571
LinearSVC 0.9095535714285714
MLPClassifier 0.9572321428571429


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. LinearSVC

In [21]:
# 가장 성능이 낮은 모델 제거
# del voting.estimators[2]

In [22]:
# model fitting
voting = VotingClassifier(estimators=[('et', ex), ('rf', rf), ('mlp',mlp)],voting = 'hard')
voting.fit(X_train, y_train)

VotingClassifier(estimators=[('et', ExtraTreesClassifier(random_state=42)),
                             ('rf', RandomForestClassifier()),
                             ('mlp', MLPClassifier())])

In [23]:
# 모델 제거 후 성능 확인 >> 성능 향상‼️
voting.score(X_val, y_val)

0.9716071428571429

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [32]:
# 랜덤포레스트 오류로 fitting 재진행
rf.fit(X_train, y_train)

RandomForestClassifier()

In [33]:
ex_pred = ex.predict(X_val)
mlp_pred = mlp.predict(X_val)
rf_pred = rf.predict(X_val)

X_val_meta_labels = np.column_stack([rf_pred, ex_pred, mlp_pred])

X_meta_train = X_val_meta_labels

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [34]:
rf_meta = RandomForestClassifier()
rf_meta.fit(X_val_meta_labels, y_val)

RandomForestClassifier()

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [35]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
ex_pred = ex.predict(X_test)
rf_pred = rf.predict(X_test)
mlp_pred = mlp.predict(X_test)

X_test_meta_labels = np.column_stack([rf_pred, ex_pred, mlp_pred])

X_meta_train = X_test_meta_labels

In [36]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
pred = rf_meta.predict(X_meta_train)

In [37]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(pred, y_test)

0.9663571428571428